Assignment 06 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData
from Assignment6Specs import scaleFactors, angles, tVectors  # Import scaling, rotation, and translation data

pygame 2.6.0 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame & OpenGL.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 06: Colin Kirby")

    # Create and Set up OpenGL.
    ctx = moderngl.create_context()
    
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

In [4]:
# Load the Vertex Data for our Platform based on the hexahedron.obj.
vertex_data_platform, _ = getObjectData("6_hexahedron.obj", normal=True)

Normal computed.


In [5]:
# Load the Vertex Data for the Teapot.
vertex_data_teapot, _ = getObjectData("teapot.obj", normal=True)

Normal exists


In [6]:
# Global VAO.
vao_platform = None
vao_teapot = None
program_platform = None
program_teapot = None

In [7]:
# Upload the Vertex Data for Platform. 
def set_vert_data_platform():
    global vao_platform, program_platform

    # Convert Vertex Data to NP Array.
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    
    # Create a buff on GPU & Load Vertex Data.
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    # Compile Shader Program.
    program_platform = ctx.program(
        # Vertex Shaders.
        vertex_shader=""" 
        #version 330
        in vec3 in_position;
        in vec3 in_normal;

        // Uniform matrices for model, view, and projection transformations
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;

        out vec3 frag_normal;

        void main() {
            frag_normal = in_normal;  // Pass normal vector to fragment shader
            gl_Position = proj * view * model * vec4(in_position, 1.0);  // Apply transformation matrices
        }
        """,
        # Fragment Shaders.
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        out vec4 frag_color;

        void main() {
            // Compute color based on the normal direction
            vec3 color = normalize(frag_normal) * 0.5 + 0.5;
            frag_color = vec4(color, 1.0);
        }
        """
    )

    # Create the VAO for the Platform.
    vao_platform = ctx.vertex_array(
        program_platform,
        [
            (vbo_platform, '3f 3f', 'in_position', 'in_normal')
        ]
    )


In [8]:
# Upload Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot

    # Convert Teapot Vertex Data & Convert to NP Array.
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    
    # Create buff on GPU & load Teapot Vertex Data.
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    # Compile Shader Program.
    program_teapot = ctx.program(
        # Vertex Shaders.
        vertex_shader="""
        #version 330
        in vec3 in_position;
        in vec3 in_normal;
        in mat4 in_modelMat;  // Instance matrix for each teapot

        uniform mat4 view;
        uniform mat4 proj;

        out vec3 frag_normal;

        void main() {
            frag_normal = in_normal;  // Pass normal to fragment shader
            // Apply projection, view, and model transformation matrices to the position
            gl_Position = proj * view * in_modelMat * vec4(in_position, 1.0);
        }
        """,
        # Fragment Shaders.
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        out vec4 frag_color;

        void main() {
            // Compute fragment color based on normal direction
            vec3 color = normalize(frag_normal) * 0.5 + 0.5;
            frag_color = vec4(color, 1.0);
        }
        """
    )

    # Initialize Matrices for Transformation for All Teapots.
    model_matrices = []
    num_teapots = 100 # Teapots to Render.
    
    for i in range(num_teapots):
        model_matrix = glm.mat4(1.0)  # Initialize Matrix.
        
        # Apply Translation (Move the Teapots) based on the tVectors.
        tx, ty, tz = tVectors[i]
        model_matrix = glm.translate(model_matrix, glm.vec3(tx, ty, tz)) 
        
        # Apply Rotation (Rotate the Teapots) based on the angles.
        model_matrix = glm.rotate(model_matrix, glm.radians(angles[i]), glm.vec3(0.0, 1.0, 0.0))
        
        # Apply Scaling (Adjust Teapot Size) based on the scaleFactors.
        model_matrix = glm.scale(model_matrix, glm.vec3(scaleFactors[i]))  
        
        # Convert GLM Matrix to Python List & Add to Matrices.
        model_matrices.append(list(model_matrix))  # Store transformation matrices
    
    # Convert Matrices to NP Array.
    model_matrices_np = np.array(model_matrices).astype('float32')
    
    # Upload Transformation Matrices.
    model_mat_buffer = ctx.buffer(model_matrices_np.tobytes())
    
    # Create VAO for Teapots.
    vao_teapot = ctx.vertex_array(
        program_teapot,
        [
            (vbo_teapot, '3f 3f /v', 'in_position', 'in_normal'),
            (model_mat_buffer, '16f /i', 'in_modelMat')
        ]
    )

In [9]:
# Initialize Vertex Data.
set_vert_data_platform()
set_vert_data_teapot()

In [10]:
# Create Proj Matrix based on provided Details.
fov = 60.0  
width, height = 800, 600
aspect_ratio = width / height  
near_plane = 10.0  
far_plane = 45.0 

In [11]:
# Create Perpective Proj Matrix w/ All Details Above.
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)

In [12]:
# Calcualtes Position of Camera for Orbiting Effect.
def calculate_orbit_camera_position(time_in_seconds):
    radius = 25.0
    initial_camera_vec = glm.vec3(20.0, 10.0, 0.0) # Provided in Assignment Details.
    initial_camera_vec = glm.normalize(initial_camera_vec)

    # Scale the Direction Vector by Camera Distance.
    camera_position = radius * initial_camera_vec

    # Rotate Camera around Y-Axis. (Orbiting Effect)
    angle_in_radians = time_in_seconds
    rotation_matrix = glm.rotate(glm.mat4(1.0), angle_in_radians, glm.vec3(0.0, 1.0, 0.0))
    rotated_camera_position = rotation_matrix * glm.vec4(camera_position, 1.0)

    # Return Cam Position.
    return glm.vec3(rotated_camera_position) 

In [13]:
# Rendering Loop.
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Set up Model Matrix for Platform (Scaling & Translations)
    model_matrix_platform = glm.mat4(1.0)
    model_matrix_platform = glm.translate(model_matrix_platform, glm.vec3(0.0, 0.0, 0.0))  # Translate to lower platform
    model_matrix_platform = glm.scale(model_matrix_platform, glm.vec3(10.0, 0.05, 10.0))  # Scale to create large flat surface

    # Get Time for Calculating Cam Position.
    time_in_seconds = pygame.time.get_ticks() / 1000.0

    # Calculate Camera Position.
    eye_position = calculate_orbit_camera_position(time_in_seconds)

    # Set Up View Matrix.
    view_matrix = glm.lookAt(
        eye_position,  # Camera Position.
        glm.vec3(0.0, 0.0, 0.0),  # Look at Origin.
        glm.vec3(0.0, 1.0, 0.0)   # Up Vector.
    )

    # Render Platform.
    if vao_platform is not None and program_platform is not None:
        try:
            # Convert Matrices to Byte Format.
            model_bytes_platform = np.array(model_matrix_platform, dtype='f4').T.flatten().tobytes()
            view_bytes = np.array(view_matrix, dtype='f4').T.flatten().tobytes()
            proj_bytes = np.array(proj_matrix, dtype='f4').T.flatten().tobytes()

            # Write Transformation Matrices.
            program_platform['model'].write(model_bytes_platform)
            program_platform['view'].write(view_bytes)
            program_platform['proj'].write(proj_bytes)

            # Clear Screen.
            ctx.clear(0.1, 0.1, 0.1)
            vao_platform.render(moderngl.TRIANGLES)  # Render Platform.

        except Exception as e:
            pass 

    # Render Teapots.
    if vao_teapot is not None and program_teapot is not None:
        try:
            # Upload View and Projection Matrices.
            view_bytes = np.array(view_matrix, dtype='f4').T.flatten().tobytes()
            proj_bytes = np.array(proj_matrix, dtype='f4').T.flatten().tobytes()

            # Write View and Projection Matrices.
            program_teapot['view'].write(view_bytes)
            program_teapot['proj'].write(proj_bytes)

            # Render 100 Teapots.
            vao_teapot.render(instances=100)

        except Exception as e:
            pass

    # Swap Display Buffers.
    pygame.display.flip()

# Clean Up.
pygame.quit()